# Exploration du Dataset House Prices

**Dataset**: {{cookiecutter.dataset_rows}} maisons avec {{cookiecutter.dataset_features}} features

**Colonnes**: date, price, bedrooms, bathrooms, sqft_living, sqft_lot, floors, waterfront, view, condition, sqft_above, sqft_basement, yr_built, yr_renovated, street, city, statezip, country

**Objectif**: Prédire le prix des maisons avec les meilleures pratiques MLOps

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("📊 Exploration du dataset House Prices")
print(f"🎯 Objectif: Prédire les prix de {{cookiecutter.dataset_rows}} maisons")

## 1. Chargement des données

In [ ]:
# Tentative de chargement du dataset réel
try:
    data_train = pd.read_csv('../data/raw/data.csv')
    print(f"✅ Dataset d'entraînement chargé: {data_train.shape}")
    
    try:
        data_test = pd.read_csv('../data/raw/output.csv')
        print(f"✅ Dataset de test chargé: {data_test.shape}")
    except:
        print("⚠️ Dataset de test non trouvé")
        data_test = None
        
except FileNotFoundError:
    print("❌ Datasets non trouvés dans data/raw/")
    print("💡 Lancez d'abord: python scripts/train.py --save-sample")
    
    # Génération de données d'exemple pour la démonstration
    print("🔄 Génération de données d'exemple...")
    
    np.random.seed(42)
    n_samples = {{cookiecutter.dataset_rows}}
    
    data_train = pd.DataFrame({
        'date': pd.date_range('2022-01-01', '2023-12-31', periods=n_samples),
        'bedrooms': np.random.randint(1, 8, n_samples),
        'bathrooms': np.round(np.random.uniform(1, 5, n_samples) * 2) / 2,
        'sqft_living': np.random.randint(500, 5000, n_samples),
        'sqft_lot': np.random.randint(3000, 50000, n_samples),
        'floors': np.random.choice([1, 1.5, 2, 2.5, 3], n_samples),
        'waterfront': np.random.choice([0, 1], n_samples, p=[0.9, 0.1]),
        'view': np.random.randint(0, 5, n_samples),
        'condition': np.random.randint(1, 6, n_samples),
        'sqft_above': np.random.randint(500, 4000, n_samples),
        'sqft_basement': np.random.randint(0, 2000, n_samples),
        'yr_built': np.random.randint(1900, 2020, n_samples),
        'yr_renovated': np.random.choice([0] + list(range(1950, 2024)), n_samples),
        'street': [f"{np.random.randint(100, 9999)} Main St" for _ in range(n_samples)],
        'city': np.random.choice(['Seattle', 'Bellevue', 'Redmond', 'Kirkland'], n_samples),
        'statezip': np.random.choice(['WA 98101', 'WA 98004', 'WA 98052'], n_samples),
        'country': 'USA'
    })
    
    # Génération des prix
    data_train['price'] = (
        300000 + 
        data_train['bedrooms'] * 30000 +
        data_train['bathrooms'] * 25000 +
        data_train['sqft_living'] * 200 +
        data_train['waterfront'] * 300000 +
        data_train['view'] * 40000 +
        np.random.normal(0, 80000, n_samples)
    ).clip(lower=100000)
    
    print(f"✅ Données d'exemple générées: {data_train.shape}")

# Aperçu des données
if 'data_train' in locals():
    print("\n=== Aperçu du dataset ===")
    display(data_train.head())
    
    print("\n=== Types de données ===")
    print(data_train.dtypes)

## 2. Analyse exploratoire des données

In [ ]:
if 'data_train' in locals():
    print("=== Informations générales ===")
    print(f"Nombre de maisons: {len(data_train):,}")
    print(f"Nombre de features: {len(data_train.columns)}")
    print(f"Période: {data_train['date'].min()} à {data_train['date'].max()}")
    
    print("\n=== Statistiques des prix ===")
    print(f"Prix moyen: ${data_train['price'].mean():,.0f}")
    print(f"Prix médian: ${data_train['price'].median():,.0f}")
    print(f"Prix min: ${data_train['price'].min():,.0f}")
    print(f"Prix max: ${data_train['price'].max():,.0f}")
    print(f"Écart-type: ${data_train['price'].std():,.0f}")
    
    print("\n=== Valeurs manquantes ===")
    missing = data_train.isnull().sum()
    if missing.sum() > 0:
        print(missing[missing > 0])
    else:
        print("✅ Aucune valeur manquante")

In [ ]:
# Distribution des prix (variable cible)
if 'data_train' in locals():
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Histogramme
    axes[0, 0].hist(data_train['price'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 0].set_title('Distribution des prix')
    axes[0, 0].set_xlabel('Prix ($)')
    axes[0, 0].set_ylabel('Fréquence')
    
    # Boxplot
    axes[0, 1].boxplot(data_train['price'])
    axes[0, 1].set_title('Boxplot des prix')
    axes[0, 1].set_ylabel('Prix ($)')
    
    # Distribution log
    log_prices = np.log1p(data_train['price'])
    axes[1, 0].hist(log_prices, bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
    axes[1, 0].set_title('Distribution log des prix')
    axes[1, 0].set_xlabel('Log(Prix)')
    axes[1, 0].set_ylabel('Fréquence')
    
    # Prix par ville
    if 'city' in data_train.columns:
        city_prices = data_train.groupby('city')['price'].mean().sort_values(ascending=False)
        axes[1, 1].bar(range(len(city_prices)), city_prices.values)
        axes[1, 1].set_title('Prix moyen par ville')
        axes[1, 1].set_ylabel('Prix moyen')
        axes[1, 1].set_xticks(range(len(city_prices)))
        axes[1, 1].set_xticklabels(city_prices.index, rotation=45)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Analyse des features numériques
if 'data_train' in locals():
    numeric_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'sqft_above', 'sqft_basement', 'yr_built']
    numeric_cols = [col for col in numeric_cols if col in data_train.columns]
    
    print(f"=== Features numériques analysées: {len(numeric_cols)} ===")
    print(numeric_cols)
    
    # Matrice de corrélation
    corr_data = data_train[numeric_cols + ['price']].corr()
    
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(corr_data, dtype=bool))
    sns.heatmap(corr_data, mask=mask, annot=True, cmap='RdBu_r', center=0, square=True, fmt='.2f')
    plt.title('Matrice de corrélation des features numériques')
    plt.tight_layout()
    plt.show()
    
    # Corrélations avec le prix
    price_corr = corr_data['price'].sort_values(ascending=False)
    print("\n=== Corrélations avec le prix ===")
    for feature, corr in price_corr.items():
        if feature != 'price':
            print(f"{feature:15}: {corr:6.3f}")

In [ ]:
# Analyse des features catégorielles
if 'data_train' in locals():
    categorical_cols = ['waterfront', 'view', 'condition', 'city']
    categorical_cols = [col for col in categorical_cols if col in data_train.columns]
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes = axes.ravel()
    
    for i, col in enumerate(categorical_cols[:4]):
        if col == 'city':
            # Prix moyen par ville
            city_data = data_train.groupby(col)['price'].mean().sort_values(ascending=False)
            axes[i].bar(range(len(city_data)), city_data.values)
            axes[i].set_title(f'Prix moyen par {col}')
            axes[i].set_xticks(range(len(city_data)))
            axes[i].set_xticklabels(city_data.index, rotation=45)
        else:
            # Boxplot pour les autres features
            data_train.boxplot(column='price', by=col, ax=axes[i])
            axes[i].set_title(f'Prix par {col}')
    
    plt.tight_layout()
    plt.show()

## 3. Insights et recommandations

### Préparation des données (selon exigences projet):
- ✅ **Nettoyage**: Gestion valeurs manquantes par interpolation
- ✅ **Outliers**: Gestion par clipping
- ✅ **Ingénierie**: Ajout de variables dérivées

### Sélection de modèles à explorer:
- RandomForestRegressor
- GradientBoostingRegressor  
- LinearRegression

### Architecture MLOps:
- ✅ **API REST** FastAPI
- ✅ **Docker** containerisation
- ✅ **CI/CD** GitHub Actions
- ✅ **Logging structuré** (timestamp, features, prédiction, durée)

In [ ]:
# Résumé final
if 'data_train' in locals():
    print("="*60)
    print("🏠 RÉSUMÉ DE L'EXPLORATION DU DATASET")
    print("="*60)
    
    print(f"📊 Dataset: {len(data_train):,} maisons avec {len(data_train.columns)} features")
    print(f"💰 Prix moyen: ${data_train['price'].mean():,.0f}")
    print(f"🏘️ Villes: {data_train['city'].nunique() if 'city' in data_train.columns else 'N/A'}")
    print(f"📅 Période: {data_train['date'].dt.year.min()}-{data_train['date'].dt.year.max()}")
    
    # Features les plus corrélées au prix
    if 'price_corr' in locals():
        print("\n🎯 Top 5 features corrélées au prix:")
        for feature, corr in price_corr.head(6).items():
            if feature != 'price':
                print(f"   {feature}: {corr:.3f}")
    
    print("\n✅ Données prêtes pour l'entraînement des modèles!")
    print("🚀 Prochaine étape: python scripts/train.py")